In [1]:
import os
from keras import Input
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow import keras
os.environ['KERAS_BACKEND'] = 'tensorflow'
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

In [4]:
import freezing
import wrn

In [6]:
depth              = 34  # 32, if ignoring conv layers carrying residuals, which are needed for increasing filter size.
growth_rate        = 10  # Growth factor
n                  = (depth-4)//6
num_classes        = 10
img_rows, img_cols = 32, 32
img_channels       = 3
batch_size         = 128
epochs             = 200
iterations         = 45000 // batch_size
weight_decay       = 0.0005
seed = 333
def color_preprocessing(x_train, x_val, x_test):
    
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')    
    x_test = x_test.astype('float32')
    
    mean = np.mean(x_train, axis=(0,1,2))  # Per channel mean
    std = np.std(x_train, axis=(0,1,2))
    x_train = (x_train - mean) / std
    x_val = (x_val - mean) / std
    x_test = (x_test - mean) / std
    
    return x_train, x_val, x_test  

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# color preprocessing
x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)  # random_state = seed
x_train45, x_val, x_test = color_preprocessing(x_train45, x_val, x_test)

y_train45 = keras.utils.to_categorical(y_train45, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# build network
img_input = Input(shape=(img_rows,img_cols,img_channels))    
model = wrn.create_wide_residual_network(img_input, nb_classes=num_classes, N=n, k=growth_rate, dropout=0.0)
print(model.summary())
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)

# set callback
checkpointer = ModelCheckpoint('model_cont_wide_28_10_c10_best_2.hdf5', verbose=1, save_best_only=True)

# set data augmentation
print('Using real-time data augmentation.')
datagen = ImageDataGenerator(horizontal_flip=True,
        width_shift_range=0.125,height_shift_range=0.125,fill_mode='reflect') # Missing pixels replaced with reflections

datagen.fit(x_train45)

In [8]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.9:
    freezing_list.append(int(epochs*0.6))
  elif i < len(model.layers) * 0.98:
    freezing_list.append(int(epochs*0.96))
freezing_list.append(epochs)

In [9]:
freezing.continue_training(model, datagen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list,lr_schedule = [[0, 0.1],[epochs*0.3,0.02],[epochs*0.6,0.004],[epochs*0.8,0.0008]],cbks=[checkpointer], all_epochs=30,weights='/kaggle/input/weights/resnet_wide_cifar10_2_30 (1).h5', name='resnet_wide_cifar10_2')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_4
.........vars
......activation_5
.........vars
......activati

[6.040000000000006,
 0.044643579581379905,
 0.35186373194058734,
 0.34076034955963314,
 0.05374559426638748]